In [1]:
from dataclasses import dataclass

@dataclass
class PreProcessingConfig:
    # Folder directories
    image_root = "/data_vault/hexai/OAICartilage/YOLO_DATASET/Image"
    label_root = "/data_vault/hexai/OAICartilage/YOLO_DATASET/2.5D_Slices_labels"
    out_dir = "/data_vault/hexai/OAICartilage/YOLO_YOLOv11_Dataset"
    dataset_name = "2_5D-Slices"

@dataclass
class YOLOv11Training:
    data_yaml = "/data_vault/hexai/OAICartilage/YOLO_YOLOv11_Dataset/KneeMRI.yaml"
    yolo_model = "yolo11n.pt"  # YOLO model for segmentation
    run_name = "SagittalKneeJoint"
    epochs = 50

In [2]:
class KneeLocalizationClasses:
    """
    Map class names for the knee MRI dataset.
    """
    def __init__(self):
        self.classes = {
            0: "SagittalKneeJoint",  # Mapping your class label for the knee joint (adjust as necessary)
        }

    def map_label(self, view):
        """
        Map the class index to a class label. 
        This could dynamically change depending on your dataset.
        """
        return self.classes.get(view, "Unknown")

In [3]:
import os
import shutil
import yaml
from sklearn.model_selection import train_test_split

def write_yolo_dataset(dataset, image_root, label_root, out_dir, dataset_type):
    """
    Write dataset to the specified directory for training, validation, or testing.
    """
    dataset_outdir = os.path.join(out_dir, dataset_type)
#    dataset_outdir = os.path.join(out_dir, dataset_type)
    os.makedirs(os.path.join(dataset_outdir, "images"), exist_ok=True)
    os.makedirs(os.path.join(dataset_outdir, "labels"), exist_ok=True)

    for image_path in dataset:
        filename = os.path.basename(image_path)
        label_filename = filename.replace(".jpg", ".txt")  # Adjust extension if needed
        label_path = os.path.join(label_root, label_filename)

        # Copy image to the output directory
        shutil.copy(image_path, os.path.join(dataset_outdir, "images", filename))

        # Copy label if it exists
        if os.path.exists(label_path):
            shutil.copy(label_path, os.path.join(dataset_outdir, "labels", label_filename))

    return dataset_outdir


def build_yolo_dataset(image_root, label_root, out_dir, test_size=0.2):
    os.makedirs(out_dir, exist_ok=True)

    # List all images
    image_files = [os.path.join(subdir, file) for subdir, _, files in os.walk(image_root)
                   for file in files if file.endswith(('.jpg', '.jpeg', '.png', '.tif', '.bmp', '.tiff', '.webp'))]

    # Split dataset into train, valid, test
    train_files, test_files = train_test_split(image_files, test_size=test_size, random_state=42)
    valid_files, test_files = train_test_split(test_files, test_size=test_size, random_state=42)

    # Write datasets to their respective directories
    train_path = write_yolo_dataset(train_files, image_root, label_root, out_dir, dataset_type="train")
    valid_path = write_yolo_dataset(valid_files, image_root, label_root, out_dir, dataset_type="valid")
    test_path = write_yolo_dataset(test_files, image_root, label_root, out_dir, dataset_type="test")


    # Write YOLO YAML file
    write_yolo_yaml_file(train_path, valid_path, test_path, out_dir)


In [4]:
def write_yolo_yaml_file(train_path, valid_path, test_path, out_dir):
    class_labels = KneeLocalizationClasses()
    yaml_info = {
        "train": train_path,
        "val": valid_path,
        "test": test_path,
        "nc": 1,  # Number of classes (1 in your case)
        "names": [class_labels.map_label(0)]  # Using 0 as the class index for "SagittalKneeJoint"
    }

    with open(f"{out_dir}/KneeMRI.yaml", 'w') as f:
        yaml.dump(yaml_info, f)


In [5]:
from ultralytics import YOLO

def get_yolo_model(yolo_model_checkpoint):
    """
    YOLOv11 Checkpoints:
    - yolov11n.pt (Nano)
    - yolov11s.pt (Small)
    - yolov11m.pt (Medium)
    - yolov11l.pt (Large)
    - yolov11x.pt (Extra Large)
    """
    return YOLO(yolo_model_checkpoint)


In [6]:
import torch
import os

def train_yolo_model(data_yaml, yolo_checkpoint_name, epochs, run_name):
    model = get_yolo_model(yolo_checkpoint_name)

    if torch.cuda.is_available():
        model = model.cuda()

    # Check if the YAML file exists
    if os.path.exists(data_yaml):
        print("data.yaml found. Proceeding with training.")
        model.train(data=data_yaml, epochs=epochs, name=run_name)
    else:
        print("data.yaml not found. Please check the path and ensure the file exists.")


In [7]:
def main(mode):
    print("Knee MRI Segmentation Entry Point!")

    if mode == "preprocess":
        config = PreProcessingConfig()
        build_yolo_dataset(config.image_root, config.label_root, config.out_dir)
        print("Preprocessing completed!")
    
    elif mode == "yolo":
        config = YOLOv11Training()
        train_yolo_model(config.data_yaml, config.yolo_model, config.epochs, config.run_name)
        print("Training completed!")


In [8]:
if __name__ == "__main__":
    main("preprocess")  # Run preprocessing

Knee MRI Segmentation Entry Point!
Preprocessing completed!


In [9]:
main("yolo")  # Run YOLO model training

Knee MRI Segmentation Entry Point!
data.yaml found. Proceeding with training.
New https://pypi.org/project/ultralytics/8.3.98 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/data_vault/hexai/OAICartilage/YOLO_YOLOv11_Dataset/KneeMRI.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=SagittalKneeJoint, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_m

train: Scanning /data_vault/hexai/OAICartilage/YOLO_YOLOv11_Dataset/train

train: WARNING ⚠️ /data_vault/hexai/OAICartilage/YOLO_YOLOv11_Dataset/train/images/9207905_20051111_10782812_slice_053.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data_vault/hexai/OAICartilage/YOLO_YOLOv11_Dataset/train/images/9207905_20051111_10782812_slice_076.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /data_vault/hexai/OAICartilage/YOLO_YOLOv11_Dataset/train/images/9680344_20040910_10698606_slice_060.jpg: 1 duplicate labels removed


train: New cache created: /data_vault/hexai/OAICartilage/YOLO_YOLOv11_Dataset/train/labels.cache


val: Scanning /data_vault/hexai/OAICartilage/YOLO_YOLOv11_Dataset/valid/l

...

      50/50      2.21G     0.6257     0.3809      1.136          9       
                 Class     Images  Instances      Box(P          R      m

                   all        640        440      0.982      0.985      0.994      0.928



50 epochs completed in 0.452 hours.
Optimizer stripped from runs/detect/SagittalKneeJoint/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/SagittalKneeJoint/weights/best.pt, 5.5MB

Validating runs/detect/SagittalKneeJoint/weights/best.pt...
Ultralytics 8.3.80 🚀 Python-3.12.7 torch-2.6.0+cu124 CUDA:0 (Quadro RTX 8000, 48407MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      m


                   all        640        440      0.982      0.985      0.994      0.928
Speed: 0.3ms preprocess, 1.3ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/SagittalKneeJoint
Training completed!


In [10]:
from ultralytics import YOLO

# Load the trained model
model = YOLO('/data_vault/hexai/OAICartilage/runs/detect/SagittalKneeJoint/weights/best.pt')
# Evaluate on the validation set
metrics = model.val()  # evaluate model performance on the validation set

Ultralytics 8.3.80 🚀 Python-3.12.7 torch-2.6.0+cu124 CUDA:0 (Quadro RTX 8000, 48407MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /data_vault/hexai/OAICartilage/YOLO_YOLOv11_Dataset/valid/labels.cache... 

val: WARNING ⚠️ /data_vault/hexai/OAICartilage/YOLO_YOLOv11_Dataset/valid/images/9207905_20051111_10782812_slice_140.jpg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)


                   all        640        440      0.982      0.985      0.994      0.927
Speed: 0.6ms preprocess, 2.1ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/val


In [11]:
# Evaluate on the test set
metrics = model.val(data="/data_vault/hexai/OAICartilage/YOLO_YOLOv11_Dataset/KneeMRI.yaml", split="test") # This will give precision, recall, mAP, and other relevant metrics

Ultralytics 8.3.80 🚀 Python-3.12.7 torch-2.6.0+cu124 CUDA:0 (Quadro RTX 8000, 48407MiB)


val: Scanning /data_vault/hexai/OAICartilage/YOLO_YOLOv11_Dataset/test/labels... 113 ima


val: New cache created: /data_vault/hexai/OAICartilage/YOLO_YOLOv11_Dataset/test/labels.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)


                   all        160        113      0.991       0.99      0.995      0.928
Speed: 2.4ms preprocess, 2.7ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/val2


In [12]:
# Print the metrics for detailed evaluation
print("Validation Metrics:", metrics)

Validation Metrics: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f75b2fd96a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
